In [3]:
import numpy as np
import pandas as pd
idx = [0, 1, 2]
d = {'channel': pd.Series(['Channel1', 'Channel2', 'Channel3'], index=idx),
     '30-day Cost': pd.Series([1765.21, 2700., 2160.], index=idx),
     'Trials': pd.Series([9865, 1500, 1200], index=idx),
     'Success': pd.Series([812, 900, 333], index=idx),
     'Cost Min': pd.Series([882.61, 1350.00, 1080.00], index=idx),
     'Cost Max': pd.Series([2647.82, 4050.00, 3240.00], index=idx)}
df = pd.DataFrame(d)
df
#    30-day Cost  Cost Max  Cost Min  Success  Trials   channel
# 0      1765.21   2647.82    882.61      812    9865  Channel1
# 1      2700.00   4050.00   1350.00      900    1500  Channel2
# 2      2160.00   3240.00   1080.00      333    1200  Channel3

,channel,30-day Cost,Trials,Success,Cost Min,Cost Max
0,Channel1,1765.21,9865,812,882.61,2647.82
1,Channel2,2700.00,1500,900,1350.00,4050.00
2,Channel3,2160.00,1200,333,1080.00,3240.00


In [4]:
import pulp

# Create variables and model
x = pulp.LpVariable.dicts("x", df.index, lowBound=0)
mod = pulp.LpProblem("Budget", pulp.LpMaximize)

# Objective function
objvals = {idx: (1.0/(df['30-day Cost'][idx]/df['Trials'][idx]))*(df['Success'][idx]/float(df['Trials'][idx])) for idx in df.index}
mod += sum([x[idx]*objvals[idx] for idx in df.index])

# Lower and upper bounds:
for idx in df.index:
    mod += x[idx] >= df['Cost Min'][idx]
    mod += x[idx] <= df['Cost Max'][idx]

# Budget sum
mod += sum([x[idx] for idx in df.index]) == 5000.0

# Solve model
mod.solve()

# Output solution
for idx in df.index:
    print( idx, x[idx].value())
# 0 2570.0
# 1 1350.0
# 2 1080.0

print ('Objective', pulp.value(mod.objective))
# Objective 1798.70495012

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/jordivilla/miniconda3/envs/OR/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/v5/vwzdch316vzcj9qldllr58800000gp/T/d5506917bb70498fa8aa29df30c4a21e-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/v5/vwzdch316vzcj9qldllr58800000gp/T/d5506917bb70498fa8aa29df30c4a21e-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 12 COLUMNS
At line 25 RHS
At line 33 BOUNDS
At line 34 ENDATA
Problem MODEL has 7 rows, 3 columns and 9 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 1 (-6) rows, 3 (0) columns and 3 (-6) elements
0  Obj 1022.5023 Primal inf 1687.39 (1) Dual inf 0.94749893 (3)
1  Obj 1798.705
Optimal - objective value 1798.705
After Postsolve, objective 1798.705, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 1798.70495 - 1 iterations time 